In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql, magics
import calitp.magics
import branca

import shared_utils
from dla_utils import _dla_utils as dla_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown

from shared_utils import geography_utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import utils

In [3]:
#adding cal-itp id 
bbb = 300
st = 290

In [4]:
pd.set_option("display.max_columns", 100)


In [5]:
rt_sched = utils.read_data()

In [6]:
rt_sched.head()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
0,290,SamTrans,1,110,110,2022-06-29,2021-04-16,2099-01-01,48,48,1.00,Wednesday,June
1,290,SamTrans,1,110,110,2022-06-28,2021-04-16,2099-01-01,48,48,1.00,Tuesday,June
2,290,SamTrans,1,110,110,2022-06-20,2021-04-16,2099-01-01,48,19,0.40,Monday,June
3,290,SamTrans,1,110,110,2022-06-21,2021-04-16,2099-01-01,48,29,0.60,Tuesday,June
4,290,SamTrans,1,110,110,2022-06-23,2021-04-16,2099-01-01,48,24,0.50,Thursday,June


## Charting

In [7]:
import altair as alt

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

#import seaborn as sns

In [8]:
rt_sched>>count(_.month)

,month,n
0,June,1820
1,May,2130


In [9]:
utils.get_agg_pct(rt_sched,
               groupings = ['agency_name', 'calitp_itp_id', 'month'],
               sum_sched ='num_sched',
               sum_vp = 'num_vp')

,agency_name,calitp_itp_id,month,num_sched,num_vp,avg
0,SamTrans,290,June,37299,23823,0.64
1,SamTrans,290,May,38561,31422,0.81
2,Big Blue Bus,300,May,39730,32927,0.83
3,Big Blue Bus,300,June,39035,33041,0.85


In [10]:
#aggregating by date
day_pct = utils.agg_by_date(rt_sched, 'num_sched', 'num_vp')

In [11]:
day_pct

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
0,290,SamTrans,1,2022-05-01,Sunday,May,765,534,0.70
1,290,SamTrans,1,2022-05-02,Monday,May,1447,1043,0.72
2,290,SamTrans,1,2022-05-03,Tuesday,May,1447,1388,0.96
3,290,SamTrans,1,2022-05-04,Wednesday,May,1455,3,0.00
4,290,SamTrans,1,2022-05-05,Thursday,May,1445,1047,0.72
...,...,...,...,...,...,...,...,...,...
117,300,Big Blue Bus,0,2022-06-26,Sunday,June,770,498,0.65
118,300,Big Blue Bus,0,2022-06-27,Monday,June,1461,1013,0.69
119,300,Big Blue Bus,0,2022-06-28,Tuesday,June,1461,1359,0.93
120,300,Big Blue Bus,0,2022-06-29,Wednesday,June,1461,1367,0.94


In [12]:
#see if we have multiple date entries
day_pct>>count(_.service_date)>>arrange(-_.n)

,service_date,n
0,2022-05-01,2
1,2022-05-02,2
2,2022-05-03,2
3,2022-05-04,2
4,2022-05-05,2
...,...,...
56,2022-06-26,2
57,2022-06-27,2
58,2022-06-28,2
59,2022-06-29,2


## Percent by Day version 1

In [13]:
(utils.bar_chart_over_time((day_pct>>filter(_.calitp_itp_id==bbb)),
                           'service_date',
                           'pct_w_vp',
                           'pct_w_vp',
                           '%',
                           'x',
                           'Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data')).properties(width=800)



alt.Chart(...)

In [14]:
(utils.bar_chart_over_time((day_pct>>filter(_.calitp_itp_id==st)),
                           'service_date',
                           'pct_w_vp',
                           'pct_w_vp',
                           '%',
                           'x',
                           'SamTrans Percent of Scheduled Trips with RT Vehicle Position Data')).properties(width=800)

alt.Chart(...)

In [15]:
# if we want to change the color scale
(utils.bar_chart_over_time((day_pct>>filter(_.calitp_itp_id==st)),
                           'service_date',
                           'pct_w_vp',
                           'pct_w_vp',
                           '%',
                           'x',
                           'SamTrans Percent of Scheduled Trips with RT Vehicle Position Data')).encode(color = alt.Color('pct_w_vp',
                       scale=alt.Scale(range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                       legend=alt.Legend(title=(utils.labeling('pct_w_vp')),
                                         symbolLimit=10))).properties(width=800)

alt.Chart(...)

## Percent by Day Version 2

In [16]:
bar = (alt.Chart(day_pct>>filter(_.calitp_itp_id==bbb))
        .mark_bar(size=8)
        .encode(
            x=alt.X('service_date:T', title=utils.labeling('service_date'), sort=("x")),
            y=alt.Y('pct_w_vp:Q', title=utils.labeling('pct_w_vp')),
            color=alt.Color(
                'weekday',
                #scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS),
                legend=alt.Legend(title=(utils.labeling('pct_w_vp'))))
        ).properties(title= "Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data"))

# # mean line
rule = alt.Chart(day_pct>>filter(_.calitp_itp_id==bbb)
                ).mark_rule(color='red'
                           ).encode(y=alt.Y('mean(pct_w_vp):Q', axis=alt.Axis(format='%')))

(bar + rule).properties(width=800)


alt.LayerChart(...)

In [17]:
bar = (alt.Chart(day_pct>>filter(_.calitp_itp_id==st))
        .mark_bar(size=8)
        .encode(
            x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
            y=alt.Y('pct_w_vp', title=utils.labeling('pct_w_vp')),
            color=alt.Color(
                'weekday',
        #        scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS),
                legend=alt.Legend(title=(utils.labeling('pct_w_vp'))))
        ).properties(title="SamTrans Percent of Scheduled Trips with RT Vehicle Position Data"))

# # mean line
rule = alt.Chart(day_pct>>filter(_.calitp_itp_id==bbb)).mark_rule(color='red').encode(
    y=alt.Y('mean(pct_w_vp)', axis=alt.Axis(format='%'))
)

(bar + rule).properties(width=800)

alt.LayerChart(...)

## Percent by Day Version 3

In [18]:
base = alt.Chart(day_pct).properties(width=550)
line = base.mark_trail().encode(
    x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
    y=alt.Y('pct_w_vp', title=utils.labeling('pct_w_vp')),
    color='agency_name',
    size='pct_w_vp:Q'
)

# mean line
rule = (alt.Chart(day_pct>>filter(_.calitp_itp_id==bbb)).mark_rule().encode(
    y=alt.Y('mean(pct_w_vp):Q', axis=alt.Axis(format='%')),
    color=alt.Color('red:N',
                    # legend=alt.Legend(values=['Average Over Time'])
                   )))

(line + rule)

alt.LayerChart(...)

## Total Number of Scheduled Trips vs Vehicle Position Trips

In [19]:
day_pct_long = day_pct.copy()
day_pct_long = day_pct_long.rename(columns={ 'total_num_sched':'Total Number Scheduled Trips',
                                            'total_num_vp':'Total Number Vehicle Position Trips'})
day_pct_long =  (day_pct_long>>select(_.agency_name,
                            _.calitp_itp_id,
                            _.service_date,
                         _['Total Number Scheduled Trips'],
                         _['Total Number Vehicle Position Trips']
                        ) 
             >> gather('measure',
                       'value',
                       _['Total Number Scheduled Trips'],
                       _['Total Number Vehicle Position Trips']
                      )
            )

In [20]:
(utils.bar_chart_over_time((day_pct_long>>filter(_.calitp_itp_id==bbb)),
                           'service_date',
                           'value', 
                           'measure',
                           ',f', 
                           'x', 
                           'Big Blus Bus Scheduled Trips with RT Vehicle Position Data')).properties(width=800)

alt.Chart(...)

In [21]:
(utils.bar_chart_over_time((day_pct_long>>filter(_.calitp_itp_id==st)),
                           'service_date',
                           'value',
                           'measure', 
                           ',f',
                           'x', 
                           'SamTrans Scheduled Trips with RT Vehicle Position Data'
                          )).properties(width=800)

alt.Chart(...)

## By Weekday

In [22]:
bbb_week = (utils.get_agg_pct((day_pct>>filter(_.calitp_itp_id==bbb)),
                  groupings = 'weekday',
                  sum_sched = 'total_num_sched',
                  sum_vp = 'total_num_vp'))

In [23]:
bbb_week

,weekday,total_num_sched,total_num_vp,avg
0,Sunday,6930,4761,0.69
1,Monday,12785,9168,0.72
2,Tuesday,13531,11969,0.88
3,Wednesday,13427,12259,0.91
4,Thursday,13425,12321,0.92
5,Friday,11707,10631,0.91
6,Saturday,6960,4859,0.70


In [24]:
 cats_day = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [25]:
#using function- x axis is not sorted
utils.bar_chart_over_time(bbb_week, 
                          'weekday', 
                          'avg', 
                          'weekday', 
                          '%', 
                          cats_day,
                          'Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data by Day').mark_bar(size=50)

alt.Chart(...)

In [26]:
bbb_week_month = utils.get_agg_pct((day_pct>>filter(_.calitp_itp_id==bbb)),
               groupings = ['agency_name', 'calitp_itp_id', 'month', 'weekday'],
               sum_sched ='total_num_sched',
               sum_vp = 'total_num_vp')

In [27]:
# add column arg

In [28]:
utils.bar_chart_over_time(bbb_week_month,
                          'weekday', 
                          'avg',
                          'weekday',
                          '%', 
                          cats_day,
                          'Average Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data by Day').encode(column='month').mark_bar(size=50)

alt.Chart(...)

In [29]:
cats_month = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [30]:
utils.bar_chart_over_time(bbb_week_month,
                          'avg',
                          'weekday',
                          'weekday',
                          '', 
                          cats_day, 
                          'Average Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data by Day').encode(x=alt.X('avg:Q', title=utils.labeling('avg'), axis=alt.Axis(format='%')),
                                                                                                                         row =alt.Row("month", sort=cats_month)).mark_bar(size=30)

alt.Chart(...)

In [31]:
(utils.bar_chart_over_time(day_pct,
                           'service_date',
                           'pct_w_vp',
                           'agency_name', 
                           '%', 
                           cats_month,
                           "Average Percent of Scheduled Trips with RT Vehicle Position Data Over Time")).mark_line(size=3).properties(width=600)

alt.Chart(...)

In [32]:
(utils.bar_chart_over_time(bbb_week_month,
                           'weekday',
                           'avg',
                           'month',
                           '%',
                           cats_day,
                           "Average Percent of Scheduled Trips with RT Vehicle Position Data by Day")).mark_line(size=3).properties(width=600)

alt.Chart(...)

### Days with percents over 95% vehicle positions data present 

In [33]:
day_pct>>filter(_.pct_w_vp>.95)

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
2,290,SamTrans,1,2022-05-03,Tuesday,May,1447,1388,0.96
5,290,SamTrans,1,2022-05-06,Friday,May,1447,1396,0.96
11,290,SamTrans,1,2022-05-12,Thursday,May,1445,1389,0.96
12,290,SamTrans,1,2022-05-13,Friday,May,1447,1377,0.95
16,290,SamTrans,1,2022-05-17,Tuesday,May,1447,1390,0.96
17,290,SamTrans,1,2022-05-18,Wednesday,May,1455,1397,0.96
18,290,SamTrans,1,2022-05-19,Thursday,May,1445,1394,0.96
19,290,SamTrans,1,2022-05-20,Friday,May,1447,1395,0.96
26,290,SamTrans,1,2022-05-27,Friday,May,1447,1381,0.95
29,290,SamTrans,1,2022-05-30,Monday,May,765,732,0.96


## Group By Month

In [34]:
avg_month = utils.groupby_twocol(day_pct, 'agency_name', 'month', 'pct_w_vp', 'month')

In [35]:
avg_month

,agency_name,month,avg
1,Big Blue Bus,May,0.81
3,SamTrans,May,0.80
0,Big Blue Bus,June,0.83
2,SamTrans,June,0.63


In [36]:
day_pct.sample()

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
33,290,SamTrans,1,2022-06-03,Friday,June,1447,1395,0.96


In [37]:
avg_month = (utils.get_agg_pct(day_pct,
                  groupings = ['agency_name', 'month'],
                  sum_sched = 'total_num_sched',
                  sum_vp = 'total_num_vp'))

In [38]:
(utils.bar_chart_over_time(avg_month, 
                           'month', 
                           'avg', 
                           'agency_name',
                           '%', 
                           cats_month, 
                           "")).encode(column='agency_name').mark_bar(size=150)

alt.Chart(...)

In [39]:
agg_weekday = (utils.get_agg_pct(day_pct,
                  groupings = ['agency_name', 'weekday'],
                  sum_sched = 'total_num_sched',
                  sum_vp = 'total_num_vp'))

In [40]:
agg_weekday

,agency_name,weekday,total_num_sched,total_num_vp,avg
0,SamTrans,Sunday,6877,4279,0.62
1,SamTrans,Monday,12083,7868,0.65
2,SamTrans,Tuesday,12765,10622,0.83
3,SamTrans,Wednesday,12821,8920,0.70
4,SamTrans,Thursday,12751,10091,0.79
5,SamTrans,Friday,11447,9077,0.79
6,SamTrans,Saturday,7116,4388,0.62
7,Big Blue Bus,Sunday,6930,4761,0.69
8,Big Blue Bus,Monday,12785,9168,0.72
9,Big Blue Bus,Tuesday,13531,11969,0.88


In [41]:
(utils.bar_chart_over_time(agg_weekday,
                           'weekday', 
                           'avg',
                           'agency_name',
                           '%', 
                           cats_month,
                           "")).encode(column='agency_name').mark_bar(size=50)

alt.Chart(...)

## Single Operator vs. All Operator Average

* all operator average: get an average pct_w_vp for each day

In [42]:
from altair.expr import datum

In [43]:
agg_all = (utils.get_agg_pct(rt_sched,
                  groupings = ['service_date'],
                  sum_sched = 'num_sched',
                  sum_vp = 'num_vp'))>>arrange(_.service_date)

In [44]:
agg_all.sample()

,service_date,num_sched,num_vp,avg
60,2022-05-06,2911,2691,0.92


In [45]:
## adding legend
#https://stackoverflow.com/questions/65226756/altair-layered-line-chart-with-legend-and-custom-colors
# scale = alt.Scale(domain=['Total Average', f'{(one_op.iloc[0]["agency_name"])} Average'], range=['#8CBCCB', '#EB9F3C'])

# total = alt.Chart(by_date).mark_line().transform_fold(
#     fold=['Total Average'], 
#     as_=['variable', 'Total Average']
# ).encode(
#     x='service_date:T', 
#     y='Total Average:Q', 
#     color=alt.Color('variable:N', scale=scale)
# )

# op = alt.Chart(by_date).mark_line().transform_fold(
#     fold=[f'{(one_op.iloc[0]["agency_name"])} Average'], 
#     as_=['variable', f'{(one_op.iloc[0]["agency_name"])} Average']
# ).encode(
#     x= alt.X('service_date:T', title = 'Service Date'),
#     y=alt.Y(f'{(one_op.iloc[0]["agency_name"])} Average:Q', title='Percent'), 
#     color=alt.Color('variable:N', scale=scale)
# )
# (total + op).properties(width=550)

# alt.Chart(by_date).mark_line().transform_fold(
#     fold=['total_average', 'operator_average'], 
#     as_=['variable', 'value']
# ).encode(
#     x=alt.X('service_date', title=utils.labeling('service_date')),
#      y=alt.Y('value:Q', title=utils.labeling('avg')),
#     color=alt.Color('variable:N', scale=scale)
# ).properties(width=550)

In [46]:
def total_average_with_1op_chart(full_df, calitp_id):
    
    agg_all = ((utils.get_agg_pct(full_df,
                  groupings = ['service_date'],
                  sum_sched = 'num_sched',
                  sum_vp = 'num_vp'))>>arrange(_.service_date)).rename(columns={'avg':'total_average'})
    
    one_op = (utils.get_agg_pct((full_df>>filter(_.calitp_itp_id==300)),
                  groupings = ['service_date', 'agency_name'],
                  sum_sched = 'num_sched',
                  sum_vp = 'num_vp'))>>arrange(_.service_date)
    
    one_op = one_op.rename(columns={'avg':f'{(one_op.iloc[0]["agency_name"])} Average'})
    
    
    by_date = pd.merge(agg_all, one_op, on= 'service_date', how='left')
    by_date = by_date.rename(columns={'total_average':'Total Average'})
    
    by_date_long =  (by_date >>select(_.service_date,
                                 _['Total Average'],
                                  _[f'{(one_op.iloc[0]["agency_name"])} Average'])
                 >>gather('measure',
                                   'value',
                                   _['Total Average'],
                                   _[f'{(one_op.iloc[0]["agency_name"])} Average']))
    by_date_long = by_date_long.rename(columns={'measure':'Averages',
                                           'value':'Percent with Vehicle Position Data'})
    
    chart = (utils.bar_chart_over_time(by_date_long, 
                           'service_date', 
                           'Percent with Vehicle Position Data', 
                           'Averages',
                           '%', 
                           "x", 
                           '')).mark_line().properties(
        title=(f'{(one_op.iloc[0]["agency_name"])} Average Compared to Overall Average')
                                                  )
    return chart.properties(width=550)

In [47]:
utils.total_average_with_1op_chart(rt_sched, 300)

alt.Chart(...)

#### Working through differing averages 
* no differences due to filtering out expired feeds
* needed to filter the calitp_deleted_at: `filter(_.calitp_deleted_at > '2022-07-01'))`

In [48]:
#starting with single date to work through the difference

In [49]:
# (utils.groupby_onecol((rt_sched>>filter(_.service_date=='2022-05-14')), 'calitp_itp_id', 'pct_w_vp'))

In [50]:
# date = rt_sched>>filter(_.service_date=='2022-05-14')

In [51]:
# (utils.groupby_onecol(date, 'calitp_itp_id', 'pct_w_vp')).avg.mean()

In [52]:
# (utils.groupby_onecol(date, 'calitp_itp_id', 'pct_w_vp'))>>summarize(avg = _.avg.mean())

In [53]:
# agg_all>>filter(_.service_date=='2022-05-14')

In [54]:
# date >>group_by(_.service_date)>>summarize(avg = _.pct_w_vp.mean())

In [55]:
# (rt_sched>>group_by(_.service_date)>>summarize(avg = _.pct_w_vp.mean()))>>filter(_.service_date=='2022-05-14')

In [56]:
# ((rt_sched>>group_by(_.calitp_itp_id, _.service_date)>>summarize(avg = _.pct_w_vp.mean()))>>filter(_.service_date=='2022-05-14'))>>summarize(avg2 = _.avg.mean())

In [57]:
# ((rt_sched>>group_by(_.agency_name, _.service_date)>>summarize(avg = _.pct_w_vp.mean()))>>filter(_.service_date=='2022-05-14'))>>summarize(avg2 = _.avg.mean())

In [58]:
# date>>group_by(_.agency_name)>>count(_.calitp_itp_id)

In [59]:
# date>>filter(_.agency_name.isnull())>>count(_.calitp_itp_id)

In [60]:
# date>>filter(_.calitp_itp_id==290,
#                  _.agency_name.isnull())

In [61]:
# (date>>filter(_.calitp_itp_id==290,
#                 _.calitp_url_number==1) >>arrange(_.route_id))

In [62]:
# (date>>filter(_.calitp_itp_id==300) >>arrange(_.route_id))

In [63]:
# rt_sched['calitp_deleted_at'] = pd.to_datetime(rt_sched['calitp_deleted_at'])

In [64]:
# date.calitp_deleted_at.info()

In [65]:
# (date >> filter(_.calitp_deleted_at > '2022-07-01'))>>filter(_.calitp_itp_id==290)>>count(_.calitp_url_number)

In [66]:
# # got it- filtering calitp_deleted_at to more recent
# date = (date >> filter(_.calitp_deleted_at > '2022-07-01'))

In [67]:
# (date>>group_by(_.calitp_itp_id)>>summarize(avg=_.pct_w_vp.mean()))

In [68]:
# (date>>group_by(_.calitp_itp_id)>>summarize(avg=_.pct_w_vp.mean())).avg.mean()

In [69]:
# (utils.groupby_onecol(date, 'service_date', 'pct_w_vp'))

## Using Functions (without DBT tables)

In [70]:
#gtfs_daily = utils.load_schedule_data(analysis_date_start, analysis_date_end, itp_id)

In [71]:
#gtfs_daily

In [72]:
#gtfs_daily.service_date.min()

In [73]:
#gtfs_daily.service_date.max()

In [74]:
#rt = utils.load_rt_data(analysis_date_start, analysis_date_end)

In [75]:
#rt

In [76]:
#rt['str_len'] = rt.trip_id.str.len()

In [77]:
#rt.str_len.value_counts()

In [78]:
#len(rt)

In [79]:
#rt.trip_id.nunique()

In [80]:
#len(gtfs_daily)

In [81]:
#gtfs_daily.trip_id.nunique()

In [82]:
#date_list =['2022-05-01','2022-05-02', '2022-05-31']

In [83]:

# from datetime import date, timedelta
# start_date = date(2022, 5, 1)
# end_date = date(2022, 5, 31)

In [84]:
# daterange = pd.date_range(start_date, end_date)

In [85]:
# (daterange)

In [86]:
#above range does not work with function

In [87]:
# dates = set()

# def daterange2(start, end):
#     for n in range(int((end - start).days) + 1):
#         yield start + timedelta(n)

# for single_date in daterange2(start_date, end_date):
#     dates.add(single_date.strftime('%Y-%m-%d'))

In [88]:
# (dates)

In [89]:
# rt['date'] = pd.to_datetime(rt['date'])

In [90]:
# gtfs_daily['service_date'] = pd.to_datetime(gtfs_daily['service_date'])

In [91]:
# single_date2 = '2022-05-08'

# (rt>>filter(_.date == single_date2))


In [92]:
# def get_pct_ran_df2(itp_id, list_of_dates):
#     pcts = []
#     for single_date in list_of_dates:
#         gtfs_daily2 = (gtfs_daily>>filter(_.service_date == single_date))
#         rt2 = (rt>>filter(_.date == single_date))
#         sched_rt_df = (pd.merge(gtfs_daily2, rt2, how='outer', on='trip_id', indicator='have_rt'))
#         #pct_ran = (utils.get_pct_ran(sched_rt_df, single_date))
#         day_pct_ran = {}
#         day_pct_ran['date'] = single_date
#         if ((len(sched_rt_df))!=0):
#             day_pct_ran['pct_trips_ran'] = ((len(sched_rt_df>>filter(_.have_rt=='both')))/(len(gtfs_daily2)))
#         elif ((len(sched_rt_df))==0):
#             day_pct_ran['pct_trips_ran'] = ''
#         pct_ran = pd.DataFrame([day_pct_ran])
#         pct_ran['n_have_rt'] = (len(sched_rt_df>>filter(_.have_rt=='both')))
#         pct_ran['n_missing_rt'] = (len(sched_rt_df>>filter(_.have_rt=='right_only')))
#         pct_ran['unmatched_rt'] = (len(sched_rt_df>>filter(_.have_rt=='left_only')))
#         pct_ran['nunique_sched'] = (gtfs_daily2.trip_id.nunique())
#         pct_ran['nunique_rt'] = (rt2.trip_id.nunique())

#         pcts.append(pct_ran)                                                    
#         #code help from: https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop
#     pcts = pd.concat(pcts)
#     pcts = pcts>>arrange(_.date)
#     return pd.DataFrame(pcts)


In [93]:
# may = utils.get_pct_ran_df(itp_id, dates, gtfs_daily, rt)

In [94]:
# may